<a href="https://colab.research.google.com/github/37stu37/OpenQuake/blob/master/OpenQuake_FFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Colab / OpenQuake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install openquake.engine

#Import libraries / paths

In [ ]:
import openquake
import configparser
import os
from pathlib import Path
import configparser

In [ ]:
folder = Path('/content/drive/My Drive/Colab Notebooks/01_Repository/OpenQuake/oq ffe/test_oqEventBased')
# files = folder / 'oqInput_EventBasedPSHA'

In [ ]:
os.chdir(folder)
!pwd

/content/drive/My Drive/Colab Notebooks/01_Repository/OpenQuake/oq ffe/test_oqEventBased


In [ ]:
!ls

expected		    __init__.py       report.rst
gsim_by_imt_logic_tree.xml  job.ini	      source_model_logic_tree.xml
gsim_logic_tree.xml	    job_ruptures.ini  source_model.xml


#Check ini file

In [ ]:
settings = configparser.ConfigParser()
settings._interpolation = configparser.ExtendedInterpolation()
settings.read(folder / 'job.ini')
# ['settings.ini']
settings.sections()
# ['SectionOne', 'SectionTwo', 'SectionThree']
settings.get('general', 'calculation_mode')

'event_based'

#Update the ini files 

In [ ]:
# settings.set('calculation', 'maximum_distance', "50")
# settings.get('calculation', 'maximum_distance')

In [ ]:
# save ini files with changes
# with open(files / 'job_Test.ini', "w") as f:
#     settings.write(f)

#Run OpenQuake

***Shell commands seems to be working using the ! symbol - might be possible to run OpenQuake directly from GoogleColab***


---



In [ ]:
# !oq engine --help

In [ ]:
!oq engine --run job.ini --exports csv